In [4]:
import sys
print(sys.executable)


c:\Program Files\Python311\python.exe


In [6]:
import numpy as np
import pandas as pd
from foundation import *

In [17]:
def obj_felipe_lucas(x, args):

    # Argumentos
    df = args[0].copy()
    n_comb = args[1]
    f_ck = args[2]
    h_z = args[3]
    n_fun = df.shape[0]

    # Variáveis de projeto
    x = np.asarray(x).reshape(n_fun, 2)   
    df['h_x (m)'] = x[:, 0]
    df['h_y (m)'] = x[:, 1]
    df['h_z (m)'] = h_z

    # Volume
    df['volume (m3)'] = df['h_x (m)'] * df['h_y (m)'] * df['h_z (m)']

    # Rótulo das combinações
    labels_comb = [f'c{i}' for i in range(1, n_comb + 1)]

    # Checagem punção
    for i in labels_comb:
        aux = f'{i}'
        df[[f'tau_sd2 - {aux}', f'tau_rd2 - {aux}', f'u_rd2 - {aux}', f'g_rd2 - {aux}', f'k_e - {aux}', f'g_ed - {aux}', f'tau_rd1 - {aux}', f'u_rd1 - {aux}', f'kx - {aux}', f'ky - {aux}', f'w_px - {aux}', f'w_py - {aux}', f'tau_sd1 - {aux}', f'g_rd1 - {aux}']] = df.apply(lambda row: verificacao_puncao_sapata(row['h_z (m)'], f_ck, row['ap (m)'], row['bp (m)'], row[f'Fz-{aux}'], row[f'Mx-{aux}'], row[f'My-{aux}']), axis=1, result_type='expand')
    df['g punção secao C'] = df[[f'g_rd2 - {i}' for i in labels_comb]].max(axis=1)
    df['g escala punção'] = df[[f'g_ed - {i}' for i in labels_comb]].max(axis=1)
    df['g punção secao Clinha'] = df[[f'g_rd1 - {i}' for i in labels_comb]].max(axis=1)

    
    # Volume final com penalizações
    df['volume final (m3)'] = df['volume (m3)'] +  df['g punção secao C'].clip(lower=0) * 1E6 + df['g escala punção'].clip(lower=0) * 1E6 + df['g punção secao Clinha'].clip(lower=0) * 1E6
    of = df['volume final (m3)'].sum()

    return of

In [ ]:
df = pd.read_excel('DADOS_TESTE_PUNCAO.xlsx')
n_comb = 3
f_ck = 25000
h_z = 1
args = args=(df, n_comb, f_ck, h_z)
x = [0.6, 0.6]

In [ ]:
of=obj_felipe_lucas(x, args)
print(of)

403698729.2021612


In [25]:
i=1 #para rodar na combinação 1

rho_x = rho_minimo_fck(f_ck)
rho_y = rho_minimo_fck(f_ck)
tau_sd2, tau_rd2, u_rd2, g_rd2, k_e, g_ed, tau_rd1, u_rd1, kx, ky, w_px, w_py, tau_sd1, g_rd1 = verificacao_puncao_sapata(h_z, f_ck, a_p=df['ap (m)'], b_p=df['bp (m)'], f_zk=df[f'Fz-c{i}'], m_xk=df[f'Mx-c{i}'], m_yk=df[f'My-c{i}'], sigma_cp=0, cob = 0.025)
#k = tabela_19_2(C1_C2)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().